# Import

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np
import pandas as pd
# import dask
import dask.dataframe as dd
import geopandas as gpd

from pathlib import Path
import sys
import os
import glob
import multiprocessing as mp
from datetime import date

from tqdm.notebook import tqdm

pd.options.mode.chained_assignment = None  # default='warn'

# Data

In [19]:
%%time
# Folders
repository = Path.cwd()
dataFolder = repository.parent.parent / 'InOutRepoData' / 'FFE'
folder = repository / 'data' / 'comparison_arcpy'

CPU times: user 163 µs, sys: 53 µs, total: 216 µs
Wall time: 200 µs


In [14]:
%%time
# Data

wind_data = pd.read_csv(repository / 'data' / 'Copy_of_GD_wind.csv')

edgelist = pd.read_parquet(repository / 'data' / 'Copy_of_edge_data.parquet', engine='pyarrow')

CPU times: user 179 ms, sys: 124 ms, total: 302 ms
Wall time: 248 ms


**data from the scenario HikWgtnMin_pfour_1-100     
scenario1 initial ignition BLDG_ID : 217, 686, 32, 166, 15, 937, 46 which translate into edge_id 216, 685, 31, 165, 14, 936, 45 (because based on index)**

**Wind speed - critical distance 45m
Wind direction NW**

**18min processing time**

In [6]:
rngdata = {'source':[216, 685, 31, 165, 14, 936, 45], 'IgnProb_bl':[1, 1, 1, 1, 1, 1, 1]} 
rngFile = pd.DataFrame(rngdata)

## Parallel computing set up

## Functions

In [10]:
# %%px

def wind_scenario(wind_data):
      import numpy as np
      i = np.random.randint(0, wind_data.values.shape[0])
      w = wind_data.values[i, 2]
      dist = wind_data.values[i, 1]
      b = wind_data.values[i, 3]
      bear_max = b + 45  # wind direction
      bear_min = b - 45
      if b == 360:
          bear_max = 45
      if b <= 0:  # should not be necessary
          bear_min = 0
      if b == 999:
          bear_max = 999
          bear_min = 0
      return bear_max, bear_min, dist # wind characteristics, bearing and distance


def ignition(rngList=rngFile, edges=edgelist):
    import numpy as np
    import pandas as pd
    rngList['rng'] = np.random.uniform(0, 1, size=rngList.values.shape[0])
    rngList = rngList[rngList['rng'] < rngList['IgnProb_bl']]
    initialIgnitions = len(rngList)
    NewActiveEdges = edges[edges['source'].isin(rngList['source'])]
    return NewActiveEdges, initialIgnitions


def mask(t, activeEdges_d, listActivatedSources_d, w_b_max, w_b_min, w_d):
    import numpy as np
    if t==0: # special case at time=0
        return activeEdges_d
    else:
        mask = (activeEdges_d.bearing.values < w_b_max) & (activeEdges_d.bearing.values < w_b_min) & (activeEdges_d.distance < w_d)
        NewActiveEdges = activeEdges_d[mask]
        NewActiveEdges = NewActiveEdges[~NewActiveEdges.source.isin(listActivatedSources_d)]
        return NewActiveEdges


def propagation(activeEdges_d, edges=edgelist):
    import numpy as np
    import pandas as pd
    NewActiveEdges = edges[edges.source.isin(activeEdges_d.target)]
    return NewActiveEdges

In [20]:
# @dview.parallel(block = False) # The @parallel decorator breaks up elementwise operations and distributes them.
def ffe_runs(n):
    import numpy as np
    import pandas as pd
    from datetime import date
    for scenario in n:
        # initial setup
        listActivatedSources = []
        listScenarioDataframes = []
        condition = True
        time = 0 
        # wind conditions
        w_bearing_max, w_bearing_min, w_distance = wind_scenario(wind_data)
        # ignition / initial state and edges selection
        ActiveEdges, numberIgnitions = ignition()
        if ActiveEdges.empty:
            print(f"no ignitions {numberIgnitions}")
            condition = False
            continue
        while condition: # spread burn zone
            ActiveEdges = mask(time, ActiveEdges, listActivatedSources, w_bearing_max, w_bearing_min, w_distance)
            if ActiveEdges.empty: #no more buildings to burn
                break
            burns = ActiveEdges.drop_duplicates(['source'], inplace=False)
#             print(f"Active edges {len(ActiveEdges)} / no duplicate = {len(burns)}")
            burns['time'] = time
            listScenarioDataframes.append(burns)
            listActivatedSources.extend(ActiveEdges.source.values)
            ActiveEdges = propagation(ActiveEdges)
            time += 1
        
        print(f'finishing scenario --- {scenario} time ---- {time} \n started with {numberIgnitions} ignitions ')

        Activations = pd.concat(listScenarioDataframes)
        Activations["scenario"] = scenario
        Activations["InitialIgnitions"] = numberIgnitions
        Activations.to_parquet(folder / f'scenario{scenario}_{str(date.today())}.parquet', engine='auto', compression="GZIP")

# Run

In [21]:
%%time
ffe_runs(range(1))

finishing scenario --- 0 time ---- 83 
 started with 7 ignitions 
CPU times: user 10.1 s, sys: 2.38 s, total: 12.4 s
Wall time: 12.5 s



### Backup and testing

In [16]:
test = pd.read_parquet(folder / 'scenario0_2020-08-06.parquet')
test

,source,target,distance,bearing,IgnProbBld,time,scenario,InitialIgnitions
58927,1339,7165,85.191863,254.643103,0.000073,0,0,8
177754,4087,1171,42.839607,146.200091,0.000068,0,0,8
1553230,36012,20709,46.135131,327.995628,0.005239,0,0,8
1838643,42567,819,61.723592,176.924230,0.000288,0,0,8
1870466,43322,69,18.925020,224.543699,0.000253,0,0,8
2437404,56553,2724,79.663636,261.571453,0.000117,0,0,8
2500477,58033,38,73.423960,28.218908,0.000313,0,0,8
3072849,71356,513,65.566426,303.012553,0.000213,0,0,8


In [14]:
# def clean_up(path=folder):
#     files = path.glob('*scenario*')
#     for f in files:
#         print(f)
#         os.remove(f)

In [15]:
# clean_up()